# 01_scrap
- scrapping data from thaigemjewelry.or.th/member and upload to DataPlatform

## Selenium login process

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4 
import requests
import pandas as pd
from bs4 import BeautifulSoup

username = "ikwamkid@outlook.com"
password = "12345678"

driver = webdriver.Chrome(executable_path=r'C:\Users\Admin\Documents\GitHub\tryme_selenium\chromedriver\chromedriver')
driver.get('https://www.thaigemjewelry.or.th/member?page=1')

# กดปุ่ม log in
login = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/a').click()

# ป้อน username/password
time.sleep(2)
user = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/form/div/div[1]/input')
pas = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/form/div/div[2]/input')
time.sleep(2)
user.send_keys(username)
pas.send_keys(password + Keys.ENTER)

# กลับไปกดเพจ member directory
time.sleep(2)
member = driver.find_element_by_xpath('/html/body/nav/div/div[3]/ul/li[3]/a').click()
memberdir = driver.find_element_by_xpath('/html/body/nav/div/div[3]/ul/li[3]/ul/li[1]/a').click()

#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.Transform='scale(0.1)';")
driver.execute_script('document.body.style.TransformOrigin = "0 0";')

# ปิด cookie policy
cookie_button = driver.find_element_by_xpath('/html/body/div[20]/button').click()

## Scraping Process

In [2]:
name = []
product_type = []
bussiness_type = []

data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup

for i in range(0,77):
    
    shop = soup.select("body > section > div > div.row > div") #css selector
    for sp in shop:
        nm = sp.select_one("body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > h5").text.strip()
        name.append(nm)

        pt = sp.select_one("body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > div.cate-type.mb-2 > p").text.strip()
        product_type.append(pt)
        
        bt = sp.select_one("body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > div:nth-child(3) > p").text.strip()
        bussiness_type.append(bt)
        
        # records.append([name, product_type, bussiness_type])



In [3]:
table = pd.DataFrame([name,product_type,bussiness_type]).transpose()
table.columns = ['name','product_type','bussiness_type']
table

,name,product_type,bussiness_type
0,"Ressigeac Gems Co.,Ltd.",Gemstones,"Wholesaler, Exporter"
1,"Yu Feng Jewelry Co.,Ltd.",Gemstones,"Retailer, Wholesaler, Importer, Exporter"
2,"Emerald Mines Co.,Ltd.",Gemstones,"Retailer, Wholesaler, Importer, Exporter"
3,"La Siam Gems Inter Co.,Ltd.","Gemsotnes,Precious and Semi Precious Stones","Exporter, Importer, Wholesaler, Retailer"
4,American Gemstone Ltd.,"Equipment and Tools,3D Printer & Software,3D P...","3D Service,3D Service"
...,...,...,...
1535,"Progold (Thailand) Co.,Ltd.","Precious Metals,Master Alloy","Agent, Wholesaler, Importer"
1536,"PATERSON JEWELLERY CO.,LTD.","Gold Jewelry,White Gold,Silver Jewelry,Manu,OE...","Jewelry Manufacturer, OEM (Original Equipment ..."
1537,"Creative Jewelry Manufacturing Co.,Ltd.","Certified Diamond : Fancy shape,Parcel Diamond...","Importer,Exporter"
1538,RG CRYSTALS LTD.,Synthetic Stones,"Retailer, Wholesaler, Importer, Exporter"


- dump data to json file

In [4]:
import json
thaigem = []

In [5]:
table = table.to_dict('records')
#print (table)

In [6]:
print (f"page:{len(table)}")

page:1540


In [7]:
with open('thaigem.json', 'w') as f:
    json.dump(table, f)

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_rawData_id = 87

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_rawData_id, file_path='thaigem.json')